<a href="https://colab.research.google.com/github/HungPham2002/Vietnamese-Handwriting-Recognition-by-using-CRNN-with-CTC-Loss/blob/main/CRNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
pwd

'/content'

In [28]:
%cd

/root


In [29]:
import tensorflow as tf
import os

# Load Data

In [1]:
# RAW_FOLDER = '/content/drive/MyDrive/UIT/Thị giác máy tính nâng cao/advanced CV/data/VN_handwritten_images'
RAW_FOLDER = '/content/drive/MyDrive/advanced CV/data/VN_handwritten_images'

In [2]:
# TRAIN_JSON = '/content/drive/MyDrive/UIT/Thị giác máy tính nâng cao/advanced CV/data/VN_handwritten_images/labels.json'
TRAIN_JSON = '/content/drive/MyDrive/advanced CV/data/VN_handwritten_images/labels.json'

In [3]:
import json
# open these label jsons files
with open(TRAIN_JSON, 'r', encoding='utf8') as f:
    train_labels = json.load(f)

In [4]:
# TEST_JSON = '/content/drive/MyDrive/UIT/Thị giác máy tính nâng cao/advanced CV/data/test_Hung/labels.json'
TEST_JSON = '/content/drive/MyDrive/advanced CV/data/test_Hung/labels.json'

In [5]:
# open these label jsons files
with open(TEST_JSON, 'r', encoding='utf8') as f:
    test_labels = json.load(f)

In [6]:
# test_labels

Here are the labels for each image!

In [7]:
#find all characters in labels (so we don't blindly add chacters not even existed in our dataset)
char_list= set()
for label in train_labels.values():
    char_list.update(set(label))
print(char_list)
char_list=sorted(char_list)
print(char_list)
len(char_list)

{'ê', 'z', 'ỡ', 'ễ', 'ọ', 'ặ', 'ể', 'ị', 'D', 'i', 'ư', 'Ứ', 'â', 'ẵ', 'ò', 'á', 'ả', 'L', '-', 'ấ', 'Ê', '6', 'ù', 'ý', '#', 'ó', 'Q', 'G', 'ì', 'E', 'ồ', 'ừ', 'õ', 'ố', ',', 'I', 'ằ', 'ơ', 'ữ', 'R', 'c', '4', '8', 'é', 'ự', 'U', '.', 'r', 'k', 'Ô', 'v', '7', 'w', '+', 'ô', 'B', 'ú', 'Ơ', "'", 'ợ', 'ộ', 'u', 'ă', '/', 'S', 'ụ', 'K', 'ế', 'ỉ', 'W', 'F', '5', 'C', 'N', '0', 'Đ', 'o', 'ử', ')', 'X', 'ờ', 'J', 'è', 'ỷ', 'ỹ', 'ệ', 'ề', ' ', 'ũ', 'M', ':', 'b', 'ĩ', 'n', 'e', 'í', 's', 't', 'x', 'ạ', 'ỵ', 'Y', 'ẽ', 'ầ', 'p', 'ở', 'ủ', '3', 'a', '9', '2', 'đ', 'à', '1', 'H', 'ậ', 'g', 'ổ', 'd', 'ã', 'ỗ', 'A', 'y', 'm', 'P', 'Â', 'ẻ', 'ỏ', 'h', 'ớ', 'q', '(', 'l', 'T', 'ẩ', 'O', 'V', 'ắ', 'ỳ', 'ứ'}
[' ', '#', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p'

140

In [8]:
# show all possible labels characters
"".join(char_list)

" #'()+,-./0123456789:ABCDEFGHIJKLMNOPQRSTUVWXYabcdeghiklmnopqrstuvwxyzÂÊÔàáâãèéêìíòóôõùúýăĐđĩũƠơưạảấầẩậắằẵặẻẽếềểễệỉịọỏốồổỗộớờởỡợụủỨứừửữựỳỵỷỹ"

In [9]:
char_list[32]

'L'

In [10]:
# convert the words to array of indexs based on the char_list
def encode_to_labels(txt):
    # encoding each output word into digits of indexes
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print("No found in char_list :", char)

    return dig_lst

In [11]:
encode_to_labels("Life is hard")

No found in char_list : f


[32, 53, 50, 0, 53, 62, 0, 52, 46, 61, 49]

In [12]:
import pathlib

In [13]:
# find all image paths for train folder 1 and 2 and test folder
train_image_path = []

for item in pathlib.Path(RAW_FOLDER).glob('**/*'):
    if item.is_file() and item.suffix not in [".json"]:
        train_image_path.append(str(item))

In [14]:
# TEST_FOLDER = '/content/drive/MyDrive/UIT/Thị giác máy tính nâng cao/advanced CV/data/test_Hung/'
TEST_FOLDER = '/content/drive/MyDrive/advanced CV/data/test_Hung'

In [15]:
# find all image paths for train folder 1 and 2 and test folder
test_data_path = []

for item in pathlib.Path(TEST_FOLDER).glob('**/*'):
    if item.is_file() and item.suffix not in [".json"]:
        test_data_path.append(str(item))

In [16]:
# test_data_path

In [17]:
# test_data_path

In [18]:
RAW_FOLDER

'/content/drive/MyDrive/advanced CV/data/VN_handwritten_images'

In [19]:
len(train_image_path)

1838

In [20]:
# train_image_pathtrain_image_path.sort()
# train_image_path

# Preprocessing




Continue to preprocess dataset and create CRNN model with CTC loss!



In [30]:
#Matching full absoluate paths and labels instead of filenames and labels
test_dict_filepath_label={}
test_data_path = pathlib.Path(os.path.join(TEST_FOLDER))
for item in test_data_path.glob('**/*.*'):
    file_name=str(os.path.basename(item))
    if file_name != "labels.json":
      label = test_labels[file_name]
      test_dict_filepath_label[str(item)]=label

In [31]:
# test_dict_filepath_label

In [32]:
#Matching full absoluate paths and labels instead of filenames and labels
dict_filepath_label={}
raw_data_path = pathlib.Path(os.path.join(RAW_FOLDER))
for item in raw_data_path.glob('**/*.*'):
    file_name=str(os.path.basename(item))
    if file_name != "labels.json":
      label = train_labels[file_name]
      dict_filepath_label[str(item)]=label

In [33]:
# dict_filepath_label

In [34]:
# find the maximum label length
test_label_lens= []
for label in test_dict_filepath_label.values():
    test_label_lens.append(len(label))
max_test_label_len = max(test_label_lens)

max_test_label_len

69

In [35]:
# find the maximum label length
label_lens= []
for label in dict_filepath_label.values():
    label_lens.append(len(label))
max_label_len = max(label_lens)

max_label_len

69

In [36]:
max_label_len = 69

In [37]:
test_image_paths = list(test_dict_filepath_label.keys())

In [38]:
# test_image_paths

In [39]:
all_image_paths = list(dict_filepath_label.keys())

In [40]:
import cv2

In [41]:
# find all widths and heights of images (this is useful if our dataset images got different sizes)
widths = []
heights = []
for image_path in all_image_paths:
    img = cv2.imread(image_path)
    (height, width, _) = img.shape
    heights.append(height)
    widths.append(width)

In [42]:
min_height = min(heights)
max_height = max(heights)
min_width = min(widths)
max_width = max(widths)

In [43]:
# this information is useful for making decision for padding and resizing
(min_height, max_height, min_width, max_width)

(94, 376, 955, 2694)

# Training

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
test_size = 0.2
train_image_paths, val_image_paths = train_test_split(all_image_paths, test_size=test_size, random_state=42)

In [46]:
len(train_image_paths)

1470

In [47]:
len(val_image_paths)

368

In [48]:
# Constants (we choose this number as we later discover that the output of our CNN is (1,9,512) from out input)
TIME_STEPS = 240

In [49]:
import numpy as np

BUILD THE PIPELINE FOR THE IMAGES, definitely this is super basic pipeline and can be improved

In [50]:
# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
resize_max_width=0
i=0
for train_img_path in train_image_paths:
    # print(f_name)
    # read input image and convert into gray scale image
    img = cv2.cvtColor(cv2.imread(train_img_path), cv2.COLOR_BGR2GRAY)

    height, width = img.shape

    # in this dataset, we don't need to do any resize at all here.
    img = cv2.resize(img,(int(118/height*width),118))

    height, width = img.shape

    if img.shape[1] > resize_max_width:
        resize_max_width = img.shape[1]
    #  thêm giá trị đệm vào hình ảnh. Điều này làm cho chiều rộng của hình ảnh bằng 2167
    # nếu chiều rộng ban đầu nhỏ hơn 2167. Giá trị đệm được tính bằng phương pháp trung vị ('median').
    img = np.pad(img, ((0,0),(0, 2167-width)), 'median')

    # Blur it sử dụng bộ lọc Gaussian để làm mờ hình ảnh.
    # Điều này giúp làm giảm nhiễu và làm mịn các đường viền trong hình ảnh.
    img = cv2.GaussianBlur(img, (5,5), 0)

    # Threshold the image using adapative threshold
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 4)

    # add channel dimension
    img = np.expand_dims(img , axis = 2)

    # Normalize each image
    img = img/255.

    label = dict_filepath_label[train_img_path]
    #print(label)
    # split data into validation and training dataset as 10% and 90% respectively
    orig_txt.append(label)
    train_label_length.append(len(label))

    # our time steps for valid input
    train_input_length.append(TIME_STEPS)
    training_img.append(img)

    # convert words to digits based on charlist
    training_txt.append(encode_to_labels(label))
    i+=1
    if (i%500 == 0):
        print ("has processed trained {} files".format(i))
print(resize_max_width)

has processed trained 500 files
has processed trained 1000 files
2167


In [51]:
############
# lists for test dataset
test_img = []
test_txt = []
test_input_length = []
test_label_length = []
test_orig_txt = []

i=0

for test_img_path in test_image_paths:
    # print(f_name)
    # read input image and convert into gray scale image
    img = cv2.cvtColor(cv2.imread(test_img_path), cv2.COLOR_BGR2GRAY)

    # in this dataset, we don't need to do any resize at all here.
    img = cv2.resize(img,(int(118/height*width),118))

    if img.shape[1] > resize_max_width:
        resize_max_width = img.shape[1]

    img = np.pad(img, ((0,0),(0, 2167-width)), 'median')

    # Blur it
    img = cv2.GaussianBlur(img, (5,5), 0)

    # Threshold the image using adapative threshold
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 4)

    # add channel dimension
    img = np.expand_dims(img , axis = 2)

    # Normalize each image
    img = img/255.

    label = test_dict_filepath_label[test_img_path]

    test_orig_txt.append(label)
    test_label_length.append(len(label))

    # our time steps for valid input
    test_input_length.append(TIME_STEPS)
    test_img.append(img)
    test_image = np.array(test_img)
    # convert words to digits based on charlist
    test_txt.append(encode_to_labels(label))
    i+=1
    if (i%500 == 0):
        print ("has processed test {} files".format(i))
print(resize_max_width)

No found in char_list : ẫ
No found in char_list : ẹ
has processed test 500 files
2167


In [52]:
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []

i=0

for val_img_path in val_image_paths:
    # print(f_name)
    # read input image and convert into gray scale image
    img = cv2.cvtColor(cv2.imread(val_img_path), cv2.COLOR_BGR2GRAY)

    # in this dataset, we don't need to do any resize at all here.
    img = cv2.resize(img,(int(118/height*width),118))

    if img.shape[1] > resize_max_width:
        resize_max_width = img.shape[1]

    img = np.pad(img, ((0,0),(0, 2167-width)), 'median')

    # Blur it
    img = cv2.GaussianBlur(img, (5,5), 0)

    # Threshold the image using adapative threshold
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 4)

    # add channel dimension
    img = np.expand_dims(img , axis = 2)

    # Normalize each image
    img = img/255.

    label = dict_filepath_label[val_img_path]

    valid_orig_txt.append(label)
    valid_label_length.append(len(label))

    # our time steps for valid input
    valid_input_length.append(TIME_STEPS)
    valid_img.append(img)

    # convert words to digits based on charlist
    valid_txt.append(encode_to_labels(label))
    i+=1
    if (i%500 == 0):
        print ("has processed test {} files".format(i))
print(resize_max_width)

2167


In [53]:
# this is the most controversial part when our max_label_len should be set different or slightly smaller than TIME_STEPS
# but let stick to the conventional/normal way: we should to be the same with our TIME_STEPS
max_label_len = TIME_STEPS

In [54]:
# import padding library
from tensorflow.keras.preprocessing.sequence import pad_sequences

# pad each output label to maximum text length, remember we did that so that we keep training with rnn consistent?
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = 0)
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = 0)

# Build Model

In [58]:
# import our model, different layers and activation function
from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [59]:
# OUR FULL MODEL OF CRNN AND LSTM

# input with shape of height=32 and width=128
inputs = Input(shape=(118,2167,1))

# Block 1
x = Conv2D(64, (3,3), padding='same')(inputs)
x = MaxPool2D(pool_size=3, strides=3)(x)
x = Activation('relu')(x)
x_1 = x

# Block 2
x = Conv2D(128, (3,3), padding='same')(x)
x = MaxPool2D(pool_size=3, strides=3)(x)
x = Activation('relu')(x)
x_2 = x

# Block 3
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x_3 = x

# Block4
x = Conv2D(256, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Add()([x,x_3])
x = Activation('relu')(x)
x_4 = x

# Block5
x = Conv2D(512, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x_5 = x

# Block6
x = Conv2D(512, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = Add()([x,x_5])
x = Activation('relu')(x)

# Block7
x = Conv2D(1024, (3,3), padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(3, 1))(x)
x = Activation('relu')(x)

# pooling layer with kernel size (2,2) to make the height/2 #(1,9,512)
x = MaxPool2D(pool_size=(3, 1))(x)

# # to remove the first dimension of one: (1, 31, 512) to (31, 512)
squeezed = Lambda(lambda x: K.squeeze(x, 1))(x)

# # # bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(blstm_1)

# # this is our softmax character proprobility with timesteps
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time

act_model = Model(inputs, outputs)

In [52]:
act_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 118, 2167,   0           []                               
                                1)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 118, 2167, 6  640         ['input_1[0][0]']                
                                4)                                                                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 39, 722, 64)  0           ['conv2d[0][0]']                 
                                                                                              

In [53]:
### ctc definition part
"""
The reason why we use ctc_batch_cost instead of ctc_cost because
In batch cost, the program managed to mask out padded sequence using label_length,
So the cost won't include the padding 0 section.
"""

"""
The way CTC is modelled currently in Keras is that you need to implement the loss function as a layer, you did that already (loss_out).
Your problem is that the inputs you give that layer are not tensors from Theano/TensorFlow but numpy arrays.
To change that one option is to model these values as inputs to your model.
That is why we need to manually create Input for labels, input length and label_length.
"""

# define the label input shape for ctc
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')

"""
where input_length and label_length are constants you created previously
the easiest way here is to have a fixed batch size in training
the lengths should have the same batch size (see shapes in the link for ctc_cost)
"""

# define the length of input and label for ctc
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

# define a ctc lambda function to take arguments and return ctc_bach_cost
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    """
    labels: tensor (number of samples, max_string_length) containing the truth labels.
    y_pred: tensor (number of samples, time_steps, num_character_labels) containing the prediction, or output of the softmax.
    input_length: tensor (number of samples, 1) containing the sequence length for each batch item in y_pred.
    label_length: tensor (number of samples, 1) containing the sequence length for each batch item in y_true.
    """
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# out loss function (just take the inputs and put it in our ctc_batch_cost)
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [54]:
# ready ctc loss function and optimizers
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

# our callbacks hell to optimize our learning
callbacks = [
    TensorBoard(
        log_dir='./logs',
        histogram_freq=10,
        profile_batch=0,
        write_graph=True,
        write_images=False,
        update_freq="epoch"),
    ModelCheckpoint(
        filepath=os.path.join('/content/drive/MyDrive/advanced CV/model/checkpoint_weights.hdf5'),
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        verbose=1),
    EarlyStopping(
        monitor='val_loss',
        min_delta=1e-8,
        patience=20,
        restore_best_weights=True,
        verbose=1),
    ReduceLROnPlateau(
        monitor='val_loss',
        min_delta=1e-8,
        factor=0.2,
        patience=10,
        verbose=1)
]
callbacks_list = callbacks

In [55]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 118, 2167,   0           []                               
                                1)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 118, 2167, 6  640         ['input_1[0][0]']                
                                4)                                                                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 39, 722, 64)  0           ['conv2d[0][0]']                 
                                                                                            

In [56]:
# notice I convert everything to numpy array here since model can fit either tensors and numpy array
# the reason why i stick with numpy array because it is simple to code and also because it is baseline solution
# I want to use openCV (can't run with tensors because it is CPU operations)
# And if I want to use tensors then i will make sure my image preprocessing functions uses tensor compatible operations

# ready our training data
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)  # all must be equal length to T timesteps
train_label_length = np.array(train_label_length)  # different length (only the same in Captcha dataset)

# ready our validating data
valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length) # all must be equal length to T timesteps
valid_label_length = np.array(valid_label_length) # different length (only the same in Captcha dataset)



In [60]:
# UNCOMMENT 2 CELLS TO TRAINING MODEL

In [61]:
# # ready our test data to test
# test_img = np.array(test_img)
# test_input_length = np.array(test_input_length) # all must be equal length to T timesteps
# test_label_length = np.array(test_label_length) # different length (only the same in Captcha dataset)

In [62]:
# # choose batchsize and epochs

# batch_size = 2
# epochs = 100

# history = model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length],
#           y=np.zeros(len(training_img)),
#           batch_size=batch_size,
#           epochs = epochs,
#           validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]),
#           verbose = 1, callbacks = callbacks_list)

# Evaluation

## Test Evaluation

### Test one sample

In [60]:
# load the saved best model weights
# act_model.load_weights(os.path.join('/content/drive/MyDrive/UIT/Thị giác máy tính nâng cao/advanced CV/model/model_checkpoint_weights.hdf5'))
act_model.load_weights(os.path.join('/content/drive/MyDrive/advanced CV/model/model_checkpoint_weights.hdf5'))

In [64]:
# resize_max_width = 2167

In [65]:
import time

In [66]:
import cv2
import numpy as np

test_image_path = "/content/drive/MyDrive/advanced CV/data/test_Hung/Private Test/0052_tests.png"

test_image = cv2.imread(test_image_path)

test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

test_image = cv2.resize(test_image, (int(118/height*width), 118))

if (test_image.shape[1] > resize_max_width):
  resize_max_width = test_image.shape[1]

test_image = np.pad(test_image, ((0,0),(0, 2167 - width)), 'median')

test_image = cv2.GaussianBlur(test_image, (5,5), 0)

test_image = cv2.adaptiveThreshold(test_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 4)

test_image = np.expand_dims(test_image, axis = 2)
test_image = np.expand_dims(test_image, axis = 0)

test_image = test_image / 255

In [67]:
start_time = time.time()
prediction = act_model.predict(test_image)
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1], greedy=True)[0][0])
one_predictions = []
i = 0
for x in out:
    print("predicted text = " , end = '')
    pred = ""
    for p in x:
        if int(p) != -1:
            pred += char_list[int(p)]
    print(pred)
    one_predictions.append(pred)
    i+=1
end_time = time.time()
execution_time = end_time - start_time

print("Time Execution: ", execution_time, " seconds")

1/1 [==============================] - 22s 22s/step
predicted text = 146/37 Huyỳnh Mấn Đạt, Phường 03, Quận 5, TP Hồ Chí Minh
Time Execution:  31.468175649642944  seconds


### Test entire samples

In [61]:
NO_PREDICTS = 100
OFFSET=0
test_prediction = act_model.predict(test_image)


18/18 [==============================] - 32s 490ms/step


In [62]:
test_prediction.shape

(549, 240, 141)

In [64]:
# use CTC decoder
out = K.get_value(K.ctc_decode(test_prediction, input_length=np.ones(test_prediction.shape[0])*test_prediction.shape[1],
                         greedy=True)[0][0])
# see the results
all_test_predictions =[]
i = 0
for x in out:
    print("original_text  = ", test_orig_txt[i+OFFSET])
    print("predicted text = ", end = '')
    pred = ""
    for p in x:
        if int(p) != -1:
            pred += char_list[int(p)]
    print(pred)
    all_test_predictions.append(pred)
    i+=1

original_text  =  35 Ni Sư Huỳnh Liên, Phường 10, Quận Tân Bình, TP Hồ Chí Minh
predicted text = 35 Ni Sư Huỳnh Lên, Phường 10, Quận Tân Bình, TP Hồ Chí Minh
original_text  =  Số 35/70B, KP 9, Phường Hố Nai, Thành Phố Biên Hoà, Đồng Nai
predicted text = Số 35/đ0B, KP 9, Phường Hố Nai, Thành Phố Bên Hoà, Đồng Nai
original_text  =  Số 99 phố Yên Phúc, Phường Phúc La, Quận Hà Đông, Hà Nội
predicted text = Số 99 Phố Yên Phúc, Phường Phúc La, Quận Hà Đông, Hà Nội
original_text  =  Số 187, tổ 8, ấp Tân Hòa, Xã Tân Tiến, Huyện Bù Đốp, Bình Phước
predicted text = Số 87, t 8, ấp Tân Hoà, Xã Tấn Tiún, Huyện Bù Đốp, Đình Phướnc
original_text  =  271 Huỳnh Ngọc Huệ, Phường Hoà Khê, Quận Thanh Khê, Đà Nẵng
predicted text = 271 Huỳỳnh Ngọc Huê, Phường Hòa Khê, Quận Thanh Khê, Đà Nẵng
original_text  =  Số 10, đường Lý Văn Lâm, Phường 1, Thành Phố Cà Mau, Cà Mau
predicted text = Số 10, đường Lý Vấn Lâm, Phường 1, Thành Phố Cà Mau, Cà Mau
original_text  =  Số 12, ngõ 11, phố Thái Hà, Phường Trung Liệt,

## Val set evaluation

In [ ]:
# predict outputs on validation images
NO_PREDICTS = 100
OFFSET=0
# prediction = act_model.predict(valid_img[OFFSET:OFFSET+NO_PREDICTS])
prediction = act_model.predict(valid_img)


In [ ]:
# NO_PREDICTS image results
# 9 timesteps
# 32 chars + 1 blank
prediction.shape

In [ ]:
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
# see the results
all_predictions =[]
i = 0
for x in out:
    print("original_text  = ", valid_orig_txt[i+OFFSET])
    print("predicted text = ", end = '')
    pred = ""
    for p in x:
        if int(p) != -1:
            pred += char_list[int(p)]
    print(pred)
    all_predictions.append(pred)
    i+=1

In [ ]:
#YOUR PART: WRITE TO PRINT OUT THE IMAGES IN NICE 2D ARRAY PLOT WITH ORIGINAL TEXT AND PREDICTED TEXT
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.figure(figsize=(10,200))
col=0
row=1
gs1 = gridspec.GridSpec(NO_PREDICTS, 1)
# gs1.update(wspace=0.025, hspace=0.025)
for n in range(NO_PREDICTS):
    plt.subplot(gs1[n])
    plt.imshow(valid_img[n][:,:,0], cmap="gray_r")
    plt.title(f"Label {n}: "+valid_orig_txt[n+OFFSET], fontsize=20, color="green")
    plt.xlabel(f"Prediction {n}: "+all_predictions[n+OFFSET], fontsize=20, color="red")


## Evaluation Metrics

In [65]:
# evaluate test set
evaluate = ocr_metrics(predicts=all_test_predictions,
                                  ground_truth=test_orig_txt,
                                  norm_accentuation=False,
                                  norm_punctuation=False)

test_result = "\n".join([
    "Metrics:",
    "Character Error Rate: {}".format(evaluate[0]),
    "Word Error Rate:      {}".format(evaluate[1]),
    "Sequence Error Rate:  {}".format(evaluate[2]),
])

In [66]:
print(test_result)

Metrics:
Character Error Rate: 0.08803356531644808
Word Error Rate:      0.2735233391975599
Sequence Error Rate:  0.9253187613843351


In [63]:
import string
import unicodedata
import editdistance


def ocr_metrics(predicts, ground_truth, norm_accentuation=False, norm_punctuation=False):
    """Calculate Character Error Rate (CER), Word Error Rate (WER) and Sequence Error Rate (SER)"""

    if len(predicts) == 0 or len(ground_truth) == 0:
        return (1, 1, 1)

    cer, wer, ser = [], [], []

    for (pd, gt) in zip(predicts, ground_truth):

        if norm_accentuation:
            pd = unicodedata.normalize("NFKD", pd).encode("ASCII", "ignore").decode("ASCII")
            gt = unicodedata.normalize("NFKD", gt).encode("ASCII", "ignore").decode("ASCII")

        if norm_punctuation:
            pd = pd.translate(str.maketrans("", "", string.punctuation))
            gt = gt.translate(str.maketrans("", "", string.punctuation))

        pd_cer, gt_cer = list(pd.lower()), list(gt.lower())
        dist = editdistance.eval(pd_cer, gt_cer)
        cer.append(dist / (max(len(pd_cer), len(gt_cer))))

        pd_wer, gt_wer = pd.lower().split(), gt.lower().split()
        dist = editdistance.eval(pd_wer, gt_wer)
        wer.append(dist / (max(len(pd_wer), len(gt_wer))))

        pd_ser, gt_ser = [pd], [gt]
        dist = editdistance.eval(pd_ser, gt_ser)
        ser.append(dist / (max(len(pd_ser), len(gt_ser))))

    cer_f = sum(cer) / len(cer)
    wer_f = sum(wer) / len(wer)
    ser_f = sum(ser) / len(ser)

    return (cer_f, wer_f, ser_f)

In [ ]:
# Evaluate Val set
evaluate = ocr_metrics(predicts=all_predictions,
                                  ground_truth=valid_orig_txt,
                                  norm_accentuation=False,
                                  norm_punctuation=False)

e_corpus = "\n".join([
    "Metrics:",
    "Character Error Rate: {}".format(evaluate[0]),
    "Word Error Rate:      {}".format(evaluate[1]),
    "Sequence Error Rate:  {}".format(evaluate[2]),
])

In [ ]:
print(e_corpus)

We got a pretty good results with CER at 1% and WER at 5%! I'm certain if I continue to apply for other techniques, this will help to reduce these numbers down. For example, I can try add to an attention layer between my CNN and RNN layers.

## Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

As we can see, the loss for validation continue to increase for the first 6 epoches and sharply drop onwards and mostly stable all the way till epoch 80.

Things to improve for the competition:
- Resize image logics with multiple image sizes (maybe as following):
    + find min, max of height and width
    + resize to a fixed height you want
    + calculate the max width of all resized images
    + padding to all images to that max width
- Combine the logic of preprocessing of train set and test set together
- Convert them to tfdataset pipeline (note that it is challenging since OpenCV won't work with tensor)
- Rework/redesign new CRNN model to fit with the new processed images.